In [ ]:
!pip install biopython matplotlib
!apt-get install ncbi-blast+ -y

In [ ]:
from google.colab import files
uploaded = files.upload()

# Make sure users upload all necessary .fasta, .fa, .fna, and .gbk files

# Create BLAST Database and Run **BLASTn**

In [ ]:
import os
import subprocess
from pathlib import Path

# Automatically find all .fasta, .fa, .fna
fasta_files = [f for f in os.listdir() if f.endswith(('.fasta', '.fa', '.fna'))]
phage_ids = [Path(f).stem for f in fasta_files]

os.makedirs('blastdb', exist_ok=True)
os.makedirs('blast_results', exist_ok=True)

# Make databases
for fasta in fasta_files:
    db_name = f"blastdb/{Path(fasta).stem}"
    subprocess.run(["makeblastdb", "-in", fasta, "-dbtype", "nucl", "-out", db_name])

# Run BLASTn pairwise
from itertools import combinations

for g1, g2 in combinations(phage_ids, 2):
    out_file = f"blast_results/{g1}_vs_{g2}.txt"
    if not os.path.exists(out_file):
        subprocess.run([
            "blastn", "-query", f"{g1}.fasta", "-db", f"blastdb/{g2}",
            "-outfmt", "6", "-evalue", "1e-5", "-out", out_file
        ])


# **Parse GenBank + BLAST**

In [ ]:
import json
from Bio import SeqIO

# Parse GBK files
data = {}
for phage in phage_ids:
    gbk_file = f"{phage}.gbk"
    if not os.path.exists(gbk_file):
        continue

    record = SeqIO.read(gbk_file, "genbank")
    cds_list = []
    for feature in record.features:
        if feature.type == "CDS":
            start = int(feature.location.start)
            end = int(feature.location.end)
            strand = feature.location.strand
            gene = feature.qualifiers.get("gene", [""])[0]
            product = feature.qualifiers.get("product", [""])[0]
            locus = feature.qualifiers.get("locus_tag", [""])[0]
            label = gene or product or locus
            cds_list.append((start, end, strand, label))

    data[phage] = {
        "length": len(record.seq),
        "cds": cds_list,
        "links": []
    }

# Parse BLAST results
for file in os.listdir('blast_results'):
    if not file.endswith(".txt"):
        continue
    g1, g2 = file.replace(".txt", "").split("_vs_")
    if g1 not in data or g2 not in data:
        continue

    with open(f"blast_results/{file}") as f:
        for line in f:
            if line.startswith("#"):
                continue
            cols = line.strip().split("\t")
            if len(cols) < 10:
                continue
            identity = float(cols[2])
            align_len = abs(int(cols[7]) - int(cols[6]))
            if identity < 80 or align_len < 100:
                continue

            qstart, qend = sorted([int(cols[6]), int(cols[7])])
            sstart, send = sorted([int(cols[8]), int(cols[9])])
            data[g1]["links"].append({
                "target": g2,
                "qstart": qstart,
                "qend": qend,
                "sstart": sstart,
                "send": send,
                "identity": identity
            })

with open("parsed_data.json", "w") as f:
    json.dump(data, f, indent=2)


# **Plot the Comparative Genomics**

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase
from matplotlib.lines import Line2D

# Load parsed data
with open("parsed_data.json") as f:
    data = json.load(f)

phage_order = list(data.keys())

def identity_to_color(identity, identity_min=80):
    norm = Normalize(vmin=identity_min, vmax=100)
    return cm.viridis(norm(identity))

fig, ax = plt.subplots(figsize=(18, 14))
ax.set_xlim(0, max(d["length"] for d in data.values()) + 1000)
ax.set_ylim(-5 * len(phage_order), 2)
ax.tick_params(axis='y', which='both', left=False, labelleft=False)
ax.set_xlabel("Genomic position (bp)", fontsize=12)

phage_pos = {phage: -i * 5 for i, phage in enumerate(phage_order)}

# Draw genome bars
for phage in phage_order:
    y = phage_pos[phage]
    length = data[phage]["length"]
    ax.plot([0, length], [y, y], lw=6, color="gray")
    ax.text(length + 500, y, phage, va='center', ha='left', fontsize=9)

    for start, end, strand, _ in data[phage]["cds"]:
        if end - start < 50:
            continue
        color = "green" if strand == 1 else "orange"
        box_y = y + 1.0
        rect = patches.Rectangle(
            (start, box_y - 0.8), end - start, 1.6,
            linewidth=0.5, edgecolor='black', facecolor=color, alpha=0.85
        )
        ax.add_patch(rect)

# Draw links
for phage in phage_order:
    src_y = phage_pos[phage]
    for link in data[phage]["links"]:
        target = link["target"]
        tgt_y = phage_pos[target]
        if abs(phage_order.index(phage) - phage_order.index(target)) != 1:
            continue
        if phage_order.index(phage) > phage_order.index(target):
            continue

        poly = patches.Polygon([
            [link["qstart"], src_y - 0.4],
            [link["qend"], src_y - 0.4],
            [link["send"], tgt_y + 0.4],
            [link["sstart"], tgt_y + 0.4],
        ], closed=True, facecolor=identity_to_color(link["identity"]), alpha=0.5)
        ax.add_patch(poly)

# Legends
cax = fig.add_axes([0.93, 0.35, 0.015, 0.4])
norm = Normalize(vmin=80, vmax=100)
cb = ColorbarBase(cax, cmap=cm.viridis, norm=norm, orientation='vertical')
cb.set_label('BLAST Identity (%)', fontsize=10)

legend_elements = [
    Line2D([0], [0], color='green', lw=6, label='+ Strand CDS'),
    Line2D([0], [0], color='orange', lw=6, label='– Strand CDS')
]
fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, 0.015),
           ncol=2, fontsize=9, frameon=True)

fig.suptitle("Comparative Genomics", fontsize=16, y=0.97)
plt.tight_layout(rect=[0, 0.05, 0.9, 0.95])
plt.savefig("phage_plot.svg", format='svg', bbox_inches='tight')
plt.show()


# **Final Step: Download the figure**

In [ ]:
from google.colab import files
files.download("phage_plot.svg")